<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/master/api_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install xmltodict

In [3]:
import pandas as pd
import re
import sys
import requests
import json
import pprint
from bs4 import BeautifulSoup
from urllib.request import urlopen
import xmltodict
from tqdm import tqdm
import random
import pprint

In [4]:
url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML'
# url 불러오기
response = requests.get(url)
# XML -> 딕셔너리 변환후 파싱
contents = response.text
dict_contents = xmltodict.parse(contents)
totalCnt = int(dict_contents['LawSearch']['totalCnt'])

In [5]:
totalCnt

2608

In [6]:
rows = []
not_match_jo = []
changed_num = 0
changed = []
eng_cnt = 0
kor_cnt = 0
total_page = int(totalCnt/20)+1
for p in tqdm(range(30,60)):#total_page)):
  url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML' \
        + '&page=' + str(p)
  # url 불러오기
  response = requests.get(url)
  # XML -> 딕셔너리 변환후 파싱
  contents = response.text
  dict_contents = xmltodict.parse(contents)
  try:
    for content in dict_contents['LawSearch']['law']:
      name_kor = content['법령명한글']
      name_eng = content['법령명영문']
      mst = content['법령일련번호']

      eng_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=elaw' \
                + '&MST=' + str(mst)
                # + '&ID=' + str(법령ID)
      eng_response = requests.get(eng_url)
      eng_contents = eng_response.text
      eng_dict_contents = xmltodict.parse(eng_contents)

      kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
                + '&MST=' + str(mst)
                # + '&ID=' + str(법령ID)
      kor_response = requests.get(kor_url)
      kor_contents = kor_response.text
      kor_dict_contents = xmltodict.parse(kor_contents)

      # # 영어조문 개수와 한글 조문 개수 맞지 않다면 건너뛰기
      # if len(eng_dict_contents['Law']['JoSection']['Jo']) != len(kor_dict_contents['법령']['조문']['조문단위']):
      #     not_match_jo += 1
      #     print(f'영한 조문 개수 맞지 않는 법령 개수', not_match_jo, kor_dict_contents['법령']['기본정보']['법령명_한글'])
      #     continue
      # eng_cnt += len(eng_dict_contents['Law']['JoSection']['Jo'])
      # kor_cnt += len(kor_dict_contents['법령']['조문']['조문단위'])
      min_num_jo = min(len(eng_dict_contents['Law']['JoSection']['Jo']), len(kor_dict_contents['법령']['조문']['조문단위']))
      temp_cnt = 0
      print('법령명:', name_kor, ', 조문 개수:', min_num_jo, ', 페이지:', p, ', 몇번째?:', content['@id'])
      try:
        # for eng_jo, kor_jo in zip(eng_dict_contents['Law']['JoSection']['Jo'],kor_dict_contents['법령']['조문']['조문단위']):
        for ind in range(min_num_jo):
          eng_jo, kor_jo = eng_dict_contents['Law']['JoSection']['Jo'][ind], kor_dict_contents['법령']['조문']['조문단위'][ind]

          if kor_jo['조문여부']!='조문':
            continue
          # if kor_jo.get('조문가지번호'):
          #   continue

          kor_total = 0
          if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
            if type(kor_jo['항']) == dict:
              kor_total += len(kor_jo['항']['호'])
              for ho in kor_jo['항']['호']:
                if ho.get('목'):
                  kor_total += len(ho['목'])
              kor_total += 1 # 호부터 시작하면 영어에서는 또 \n이 하나 늘어남
            else:
              kor_total += len(kor_jo['항'])
              for hang in kor_jo['항']:
                if hang.get('호'):
                  kor_total += len(hang['호'])
                  for ho in hang['호']:
                    if ho.get('목'):
                      kor_total += len(ho['목'])
          else:
            kor_total = 1

          if kor_jo.get('조문참고자료'):
            if '\n\t\t\n\t\t' in kor_jo['조문참고자료']:
              kor_total += 2
              kor_jo['조문참고자료'] = kor_jo['조문참고자료'].replace('\n\t\t\n\t\t', '\n')
            else:
              kor_total += 1

          if (eng_jo['joCts'].count('\n')+1) != kor_total:
            not_match_jo.append([name_kor, kor_jo['조문제목']])
            continue

          if eng_jo['joYn']!= 'Y':
            changed_num += 1
            changed.append([eng_jo, kor_jo])
            print('안바뀐 법령 개수:', changed_num)
            continue
          # if eng_jo['joBrNo']!='00':
          #   continue

          eng_text = eng_jo['joCts']

          jo_txt = ''
          hang_txt = ''
          ho_txt = ''
          mok_txt = ''

          jo_txt += kor_jo['조문내용']
          if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
            if type(kor_jo['항']) == dict: # 바로 호내용일 때
              for ho in kor_jo['항']['호']:
                ho_txt += '\n' + ho['호내용']
                if ho.get('목'):
                  for mok in ho['목']:
                    mok_txt += '\n' + mok['목내용']
                  ho_txt += mok_txt
              hang_txt += ho_txt
            else:
              for hang in kor_jo['항']:
                if hang_txt == '':
                  hang_txt += hang['항내용'] # 영어의 경우 맨 첫항과 조문 내용 사이에 \n으로 분리가 안돼있음
                else:
                  hang_txt += '\n' + hang['항내용']

                if hang.get('호'):
                  for ho in hang['호']:
                    ho_txt += '\n' + ho['호내용']
                    if ho.get('목'):
                      for mok in ho['목']:
                        mok_txt += '\n' + mok['목내용']
                      ho_txt += mok_txt
                  hang_txt += ho_txt
              jo_txt += hang_txt
          if kor_jo.get('조문참고자료'):
            eng_text = '\n'.join([i.strip() for i in eng_text.split('\n')[:-1]]) # 조문참고자료 부분 삭제하기
          kor_text = jo_txt
          rows.append({'name_kor':name_kor, 'name_eng':name_eng, 'MST':mst, 'eng':eng_text, 'kor':kor_text})
      except Exception as e:
        print('eng_kor_contents 예외 발생:', e)
  except Exception as e:
    print('dict_contents 예외 발생:', e)
print(len(not_match_jo))
df = pd.DataFrame(rows)

  0%|          | 0/19 [00:00<?, ?it/s]

법령명: 금융회사의 지배구조에 관한 법률 , 조문 개수: 57 , 페이지: 31 , 몇번째?: 601
법령명: 금융회사의 지배구조에 관한 법률 시행령 , 조문 개수: 43 , 페이지: 31 , 몇번째?: 602
법령명: 급경사지 재해예방에 관한 법률 , 조문 개수: 48 , 페이지: 31 , 몇번째?: 603
법령명: 기간제 및 단시간근로자 보호 등에 관한 법률 , 조문 개수: 32 , 페이지: 31 , 몇번째?: 604
법령명: 기간제 및 단시간근로자 보호 등에 관한 법률 시행령 , 조문 개수: 7 , 페이지: 31 , 몇번째?: 605
법령명: 기계설비법 , 조문 개수: 40 , 페이지: 31 , 몇번째?: 606
법령명: 기능성 양잠산업 육성 및 지원에 관한 법률 , 조문 개수: 12 , 페이지: 31 , 몇번째?: 607
법령명: 기르는 어업육성법 , 조문 개수: 41 , 페이지: 31 , 몇번째?: 608
법령명: 기부금품의 모집 및 사용에 관한 법률 , 조문 개수: 18 , 페이지: 31 , 몇번째?: 609
법령명: 기부금품의 모집 및 사용에 관한 법률 시행령 , 조문 개수: 23 , 페이지: 31 , 몇번째?: 610
법령명: 기상관측표준화법 , 조문 개수: 40 , 페이지: 31 , 몇번째?: 611
법령명: 기상관측표준화법 시행령 , 조문 개수: 17 , 페이지: 31 , 몇번째?: 612
법령명: 기상법 , 조문 개수: 88 , 페이지: 31 , 몇번째?: 613
법령명: 기상법 시행령 , 조문 개수: 29 , 페이지: 31 , 몇번째?: 614
법령명: 기상산업진흥법 , 조문 개수: 38 , 페이지: 31 , 몇번째?: 615
법령명: 기상산업진흥법 시행령 , 조문 개수: 25 , 페이지: 31 , 몇번째?: 616
법령명: 기술개발촉진법 , 조문 개수: 21 , 페이지: 31 , 몇번째?: 617
법령명: 기술개발촉진법 시행령 , 조문 개수: 31 , 페이지: 31 , 몇번째?: 618
법령명: 기술보증기금법 , 조문 개수: 68 , 페

  5%|▌         | 1/19 [02:08<38:37, 128.75s/it]

법령명: 기술보증기금법 시행령 , 조문 개수: 42 , 페이지: 31 , 몇번째?: 620
법령명: 기술사법 , 조문 개수: 34 , 페이지: 32 , 몇번째?: 621
법령명: 기술사법 시행령 , 조문 개수: 33 , 페이지: 32 , 몇번째?: 622
법령명: 기술사법 시행규칙 , 조문 개수: 14 , 페이지: 32 , 몇번째?: 623
법령명: 기술신용보증기금법 시행규칙 , 조문 개수: 6 , 페이지: 32 , 몇번째?: 624
법령명: 기술의 이전 및 사업화 촉진에 관한 법률 , 조문 개수: 66 , 페이지: 32 , 몇번째?: 625
법령명: 기술의 이전 및 사업화 촉진에 관한 법률 시행령 , 조문 개수: 59 , 페이지: 32 , 몇번째?: 626


 11%|█         | 2/19 [02:57<23:04, 81.46s/it] 

...

dict_contents 예외 발생: 'NoneType' object is not subscriptable
법령명: 미성년자보호법 , 조문 개수: 9 , 페이지: 48 , 몇번째?: 941
법령명: 미성년자보호법시행령 , 조문 개수: 6 , 페이지: 48 , 몇번째?: 942
법령명: 미세먼지 저감 및 관리에 관한 특별법 , 조문 개수: 32 , 페이지: 48 , 몇번째?: 943
법령명: 미세먼지 저감 및 관리에 관한 특별법 시행령 , 조문 개수: 19 , 페이지: 48 , 몇번째?: 944
법령명: 민간인 통제선 이북지역의 산지관리에 관한 특별법 , 조문 개수: 38 , 페이지: 48 , 몇번째?: 945
법령명: 민간임대주택에 관한 특별법 , 조문 개수: 96 , 페이지: 48 , 몇번째?: 946
법령명: 민간임대주택에 관한 특별법 시행령 , 조문 개수: 83 , 페이지: 48 , 몇번째?: 947
법령명: 민ㆍ군기술협력사업 촉진법 , 조문 개수: 27 , 페이지: 48 , 몇번째?: 948
법령명: 민방위기본법 , 조문 개수: 43 , 페이지: 48 , 몇번째?: 949
법령명: 민방위기본법 시행령 , 조문 개수: 63 , 페이지: 48 , 몇번째?: 950
법령명: 민법 , 조문 개수: 1323 , 페이지: 48 , 몇번째?: 951
eng_kor_contents 예외 발생: '조문제목'
법령명: 민사소송 등에서의 전자문서 이용 등에 관한 법률 , 조문 개수: 16 , 페이지: 48 , 몇번째?: 952
법령명: 민사소송 등 인지법 , 조문 개수: 16 , 페이지: 48 , 몇번째?: 953
법령명: 민사소송법 , 조문 개수: 563 , 페이지: 48 , 몇번째?: 954
법령명: 민사소송비용법 , 조문 개수: 14 , 페이지: 48 , 몇번째?: 955
법령명: 민사조정법 , 조문 개수: 48 , 페이지: 48 , 몇번째?: 956
법령명: 민사집행법 , 조문 개수: 330 , 페이지: 48 , 몇번째?: 957
법령명: 민영교도소 등의 설치ㆍ운영

 95%|█████████▍| 18/19 [34:26<02:13, 133.67s/it]

법령명: 민원 처리에 관한 법률 , 조문 개수: 59 , 페이지: 48 , 몇번째?: 960
법령명: 민원 처리에 관한 법률 시행령 , 조문 개수: 72 , 페이지: 49 , 몇번째?: 961
법령명: 민주평화통일자문회의법 , 조문 개수: 42 , 페이지: 49 , 몇번째?: 962
법령명: 민주화운동 관련자 명예회복 및 보상 등에 관한 법률 , 조문 개수: 33 , 페이지: 49 , 몇번째?: 963
법령명: 밀산업 육성법 , 조문 개수: 24 , 페이지: 49 , 몇번째?: 964
법령명: 밀항단속법 , 조문 개수: 10 , 페이지: 49 , 몇번째?: 965
법령명: 바둑 진흥법 , 조문 개수: 13 , 페이지: 49 , 몇번째?: 966
법령명: 바르게살기운동조직 육성법 , 조문 개수: 9 , 페이지: 49 , 몇번째?: 967
법령명: 박물관 및 미술관 진흥법 , 조문 개수: 54 , 페이지: 49 , 몇번째?: 968
법령명: 박물관 및 미술관 진흥법 시행령 , 조문 개수: 27 , 페이지: 49 , 몇번째?: 969
법령명: 반도체집적회로의 배치설계에 관한 법률 , 조문 개수: 62 , 페이지: 49 , 몇번째?: 970
법령명: 반도체집적회로의 배치설계에 관한 법률 시행령 , 조문 개수: 47 , 페이지: 49 , 몇번째?: 971
법령명: 반도체집적회로의 배치설계에 관한 법률 시행규칙 , 조문 개수: 26 , 페이지: 49 , 몇번째?: 972
법령명: 발달장애인 권리보장 및 지원에 관한 법률 , 조문 개수: 55 , 페이지: 49 , 몇번째?: 973
법령명: 발명교육의 활성화 및 지원에 관한 법률 , 조문 개수: 20 , 페이지: 49 , 몇번째?: 974
법령명: 발명교육의 활성화 및 지원에 관한 법률 시행령 , 조문 개수: 14 , 페이지: 49 , 몇번째?: 975
법령명: 발명진흥법 , 조문 개수: 123 , 페이지: 49 , 몇번째?: 976
법령명: 발명진흥법 시행령 , 조문 개수: 83 , 페이지: 49 , 몇번째?: 977
법

100%|██████████| 19/19 [36:35<00:00, 115.53s/it]

법령명: 방문판매 등에 관한 법률 시행령 , 조문 개수: 78 , 페이지: 49 , 몇번째?: 980
1567


In [1]:
df.to_csv('/content/drive/MyDrive/KB/30-50.csv', index=False)

NameError: ignored

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML' \
        + '&page=1'
# url 불러오기
response = requests.get(url)
# XML -> 딕셔너리 변환후 파싱
contents = response.text
dict_contents = xmltodict.parse(contents)

content = dict_contents['LawSearch']['law'][5]

name_kor = content['법령명한글']
name_eng = content['법령명영문']
mst = content['법령일련번호']
print(name_kor)
eng_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=elaw' \
          + '&MST=' + str(mst)
          # + '&ID=' + str(법령ID)
eng_response = requests.get(eng_url)
eng_contents = eng_response.text
eng_dict_contents = xmltodict.parse(eng_contents)

kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
          + '&MST=' + str(mst)
          # + '&ID=' + str(법령ID)
kor_response = requests.get(kor_url)
kor_contents = kor_response.text
kor_dict_contents = xmltodict.parse(kor_contents)

2018 평창 동계올림픽대회 및 동계패럴림픽대회 지원 등에 관한 특별법


KeyboardInterrupt: ignored

In [ ]:
count = 0
for temp in kor_dict_contents['법령']['조문']['조문단위']:
    if temp.get('조문제목'):
        if temp['조문제목'] == '기념화폐의 판매':
            print(count)
    count += 1

17


In [17]:
eng_dict_contents['Law']['JoSection']['Jo'][6]

{'@No': '7',
 'chapNo': '00000200000000000000',
 'joNo': '0005',
 'joBrNo': '00',
 'joYn': 'Y',
 'joTtl': 'Establishment of Organizing Committee',
 'joCts': 'Article 5 (Establishment of Organizing Committee) (1) In order to conduct the following projects and activities for the preparation and successful operation of the Games, the PyeongChang Organizing Committee for the 2018 Olympic and Paralympic Winter Games (hereinafter referred to as "the Organizing Committee") shall be established upon authorization from the Minister of Culture, Sports and Tourism: <Amended by Act No. 14198, May 29, 2016>\n  1. Establishment of a comprehensive plan for the Games, and establishment and  implementation of detailed operational plans;\n  2. Establishment and implementation of a plan to establish and use Games-related facilities;\n  3. Cooperation with international sports organizations promoting the Games;\n  4. Other projects necessary for the smooth preparation and operation of the Games.\n  (2) Th

In [16]:
kor_dict_contents['법령']['조문']['조문단위'][6]

{'@조문키': '0005001',
 '조문번호': '5',
 '조문여부': '조문',
 '조문제목': '조직위원회의 설립',
 '조문시행일자': '20190401',
 '조문이동이전': None,
 '조문이동이후': None,
 '조문변경여부': 'N',
 '조문내용': '제5조(조직위원회의 설립)',
 '항': [{'항번호': '①',
   '항내용': '① 대회의 준비 및 성공적인 개최를 위한 다음 각 호의 사업과 활동을 하기 위하여 문화체육관광부장관의 인가를 받아 2018 평창 동계올림픽대회 및 동계패럴림픽대회 조직위원회(이하 "조직위원회"라 한다)를 설립한다. <개정 2016.5.29>',
   '호': [{'호번호': '1.', '호내용': '1. 대회 종합계획 수립 및 세부 운영계획의 수립ㆍ시행'},
    {'호번호': '2.', '호내용': '2. 대회관련시설의 설치ㆍ이용에 관한 계획의 수립ㆍ시행'},
    {'호번호': '3.', '호내용': '3. 대회를 주최하는 국제스포츠기구와의 협력'},
    {'호번호': '4.', '호내용': '4. 그 밖에 대회의 원활한 준비 및 운영을 위하여 필요한 사업'}]},
  {'항번호': '②', '항내용': '② 조직위원회는 법인으로 한다.'},
  {'항번호': '③',
   '항내용': '③ 조직위원회에 위원장, 부위원장, 집행위원 및 감사를 두고 이를 임원으로 하며, 최고의결기구인 위원총회 및 업무의 효율적인 추진을 위한 집행위원회를 둔다.'},
  {'항번호': '④',
   '항내용': '④ 제3항에 따른 임원의 정원, 임기 및 선출 방법, 위원총회와 집행위원회의 구성 및 기능 등에 관한 사항은 정관으로 정한다.'},
  {'항번호': '⑤',
   '항내용': '⑤ 조직위원회에 관하여 이 법에서 규정한 것 외에는 「민법」 중 재단법인에 관한 규정을 준용한다.'}],
 '조문참고자료': '[법률 제11226호(2012. 1. 26.) 제5조의 개정규정은 같은 법 부칙 제2조의 규정에 의하여

In [ ]:
# 영한 조문 개수 맞지 않는 법령 개수 2 간접투자자산 운용업법 시행령 17
# 영한 조문 개수 맞지 않는 법령 개수 3 간척지의 농어업적 이용 및 관리에 관한 법률 9
# 영한 조문 개수 맞지 않는 법령 개수 4 개발이익 환수에 관한 법률 14
# 영한 조문 개수 맞지 않는 법령 개수 5 개별소비세법 18

In [ ]:
kor_jo = kor_dict_contents['법령']['조문']['조문단위'][43]
eng_jo = eng_dict_contents['Law']['JoSection']['Jo'][43]
kor_total = 0
if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
  if type(kor_jo['항']) == dict:
    kor_total += len(kor_jo['항']['호'])
    for ho in kor_jo['항']['호']:
      if ho.get('목'):
        kor_total += len(ho['목'])
    kor_total += 1
  else:
    kor_total += len(kor_jo['항'])
    for hang in kor_jo['항']:
      if hang.get('호'):
        kor_total += len(hang['호'])
        for ho in hang['호']:
          if ho.get('목'):
            kor_total += len(ho['목'])
else:
  kor_total = 1
print(kor_total)

if kor_jo.get('조문참고자료'):
  kor_total += (1 + kor_jo['조문참고자료'].count('\n'))
print(kor_total)
if (eng_jo['joCts'].count('\n')+1) != kor_total:
  not_match_jo += 1
  print(not_match_jo)

6
9
4
